# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

## Visualization of a membrane defined by a mesh

In [ ]:
from bioexplorer import BioExplorer, Cell, Protein, Membrane, MolecularSystemAnimationParams, Vector2, Vector3, Quaternion
import glob
import os

be = BioExplorer('localhost:5000')
core = be.core_api()
status = be.reset_scene()

Future exception was never retrieved
future: <Future finished exception=ConnectionClosedError(None, None, None)>
websockets.exceptions.ConnectionClosedError: no close frame received or sent


### Resources

In [ ]:
resource_folder = os.path.abspath('../../tests/test_files')
pdb_folder = os.path.join(resource_folder, 'pdb')
obj_folder = os.path.join(resource_folder, 'obj')
membrane_folder = os.path.join(pdb_folder, 'membrane')
lipids_folder = os.path.join(membrane_folder, 'lipids')
transporters_folder = os.path.join(pdb_folder, 'transporters')

representation = be.protein_representation.ATOMS_AND_STICKS

### Configuration

In [ ]:
mesh_source= os.path.join(obj_folder, 'suzanne.obj')
rotation = Quaternion(1, 0, 0, 0)
position = Vector3(100.0, 0.0, 0.0)
scale = Vector3(2.5,2.5,2.5)

## Camera position

In [ ]:
status = be.core_api().set_camera(
    orientation=[-0.114, 0.341, 0.020, 0.932],
    position=[368.152, 107.242, 367.312],
    target=[50.364, -6.824, -5.159]
)

### Membrane

In [ ]:
name = 'Suzanne'

# ACE2 receptor definition
ace2_receptor = Protein(
    name=be.NAME_TRANS_MEMBRANE + '_ACE2',
    source=os.path.join(pdb_folder, '6m18.pdb'),
    transmembrane_params=Vector2(1.0, 2.0),
    rotation=Quaternion(0.0, 1.0, 0.0, 0.0),
    animation_params=MolecularSystemAnimationParams(1), occurrences=20)

# GLUT3 definition
transporter = Protein(
    name=be.NAME_TRANS_MEMBRANE + '_GLUT3',
    source=os.path.join(transporters_folder, '4zwc.pdb'),
    transmembrane_params=Vector2(1.0, 2.0),
    rotation=Quaternion(0.707, 0.707, 0.0, 0.0),
    animation_params=MolecularSystemAnimationParams(2), chain_ids=[1], occurrences=30)

# Membrane definition
pdb_lipids = glob.glob(os.path.join(lipids_folder, '*.pdb'))[:8]

membrane = Membrane(
    lipid_sources=pdb_lipids, lipid_density=1.0,
    load_non_polymer_chemicals=True, load_bonds=True,
    animation_params=MolecularSystemAnimationParams(0, 1, 0.025, 2, 0.5)
)

clipping_planes = [
    [0.0, 1.0, 0.0, 20],
    [1.0, 0.0, 0.0, 10],
]

# Cell definition
cell = Cell(
    name=name,
    shape=be.assembly_shape.MESH,
    shape_params=scale,
    shape_mesh_source=mesh_source, 
    membrane=membrane,
    proteins=[ace2_receptor, transporter]
)

# Add cell to scene
status = be.add_cell(
    cell=cell, representation=representation,
    clipping_planes=clipping_planes
)

## Rendering settings

In [ ]:
status = be.set_rendering_quality(be.rendering_quality.HIGH)

## Environment with protein location constraints

In [ ]:
from bioexplorer import Protein, Volume, MolecularSystemAnimationParams

In [ ]:
scene_size = Vector3(300, 300, 300)
nb_glucoses = 100
nb_lactoferrins = 20

glucose_path = os.path.join(pdb_folder, 'glucose.pdb')
immune_folder = os.path.join(pdb_folder, 'immune')
lactoferrin_path = os.path.join(immune_folder, '1b0l.pdb')
defensin_path = os.path.join(immune_folder, '1ijv.pdb')

In [ ]:
constraints=list()
constraints.append([be.position_constraint.OUTSIDE, 'Suzanne'])

In [ ]:
glucose = Protein(
    name=be.NAME_GLUCOSE, 
    source=glucose_path, load_non_polymer_chemicals=True, 
    animation_params=MolecularSystemAnimationParams(1, 0, 0.0, 2, 1.0),
    occurrences=nb_glucoses)
volume = Volume(
    name=be.NAME_GLUCOSE, 
    shape=be.assembly_shape.CUBE, shape_params=scene_size,
    protein=glucose)
status = be.add_volume(
    volume=volume,
    representation=be.protein_representation.ATOMS_AND_STICKS,
    atom_radius_multiplier=2.0, constraints=constraints)

In [ ]:
lactoferrin = Protein(
    name=be.NAME_LACTOFERRIN, 
    source=lactoferrin_path, load_non_polymer_chemicals=True,
    animation_params=MolecularSystemAnimationParams(3, 0, 0.0, 4, 1.0),
    occurrences=nb_lactoferrins)
lactoferrins_volume = Volume(
    name=be.NAME_LACTOFERRIN, 
    shape=be.assembly_shape.CUBE, shape_params=scene_size,
    protein=lactoferrin,
)
status = be.add_volume(
    volume=lactoferrins_volume,
    representation=be.protein_representation.ATOMS_AND_STICKS,
    atom_radius_multiplier=2.0, constraints=constraints)

In [ ]:
be.apply_default_color_scheme(shading_mode=be.shading_mode.BASIC)

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

In [ ]:
mm.create_snapshot(
    renderer='advanced',
    path='/tmp', base_name='suzanne',
    size=[512, 512], samples_per_pixel=64)